In [1]:
import os
import pytesseract
import pandas as pd

from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

try:
    from PIL import Image
except ImportError:
    import Image
    

In [2]:
def ocr_core(filename):
    """
    This function will handle the core OCR processing of images.
    """
    text = pytesseract.image_to_string(Image.open(filename))  
    # We'll use Pillow's Image class to open the image and pytesseract to detect the string in the image
    
    return text


In [3]:
# instantiate a chrome options object so you can set the size and headless preference
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920x1080")

# download the chrome driver from https://sites.google.com/a/chromium.org/chromedriver/downloads and put it in the
# current directory
chrome_driver = os.getcwd() +"\\chromedriver.exe"

# go to Google and click the I'm Feeling Lucky button
driver = webdriver.Chrome(chrome_options=chrome_options, executable_path=chrome_driver)


C:\Continuum\Anaconda3\envs\van\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()


In [4]:
date_ = datetime.today().strftime('%Y%m%d')
date_

'20191224'

In [5]:
os.getcwd()

'C:\\Continuum\\Anaconda3\\envs\\van\\notebooks\\stock'

In [6]:
actual_files_dir = os.getcwd() + '\\my_actual\\'
mini_files_dir = os.getcwd() + '\\my_mini\\'

if not os.path.exists(actual_files_dir):
    os.makedirs(actual_files_dir)
    
if not os.path.exists(mini_files_dir):
    os.makedirs(mini_files_dir)

actual_directory = actual_files_dir + date_
mini_directory = mini_files_dir + date_

if not os.path.exists(actual_directory):
    os.makedirs(actual_directory)
    
if not os.path.exists(mini_directory):
    os.makedirs(mini_directory)


### Performance

In [22]:
file = os.getcwd() + '\\Lock-Stock.xlsx'
lock_stock = pd.read_excel(file, sheet_name='LockStockBarrel')
lock_inactive = pd.read_excel(file, sheet_name='Inactive')
lock_stock.head()

,#,Share,Qty,Buy \nPrice,Buy\nAmt,Present \nPrice,Present \nValue,Diff \nPrice,Diff \nPrice (%),Profit,...,Totem,Plan \nQty,Lien,Total \nQty,New Avg. \nPrice,Avg Diff \nPrice \n(%),New Buy \nValue,Future \nPrice,Future \nValue,Future \nProfit
0,1.0,AB CAPITAL,40.0,88.39,3535.6,98.00,3920.0,9.61,10.87,384.4,...,384.4,10.0,980.00,50.0,90.31,0.08,4515.50,109.27,5463.50,948.00
1,2.0,ENGINEERS INDIA,30.0,106.38,3191.4,103.50,3105.0,-2.88,-2.71,-86.4,...,-63.9,8.0,828.00,38.0,105.77,-0.02,4019.26,115.40,4385.20,365.94
2,3.0,IFCI,200.0,17.43,3486.0,6.45,1290.0,-10.98,-62.99,-2196.0,...,-1851.5,50.0,322.50,250.0,15.23,-1.36,3807.50,7.19,1797.50,-2010.00
3,4.0,ITC,10.0,247.25,2472.5,244.35,2443.5,-2.90,-1.17,-29.0,...,-29.0,3.0,733.05,13.0,246.58,-0.01,3205.54,272.45,3541.85,336.31
4,5.0,JSL,100.0,36.80,3680.0,37.95,3795.0,1.15,3.13,115.0,...,115.0,25.0,948.75,125.0,37.03,0.02,4628.75,42.31,5288.75,660.00


In [23]:
lock_stock_req = lock_stock[pd.notnull(lock_stock['Share'])]
len(lock_stock_req)

13

In [24]:
my_shares = lock_stock_req['Share'].unique().tolist()
my_shares

['AB CAPITAL',
 'ENGINEERS INDIA',
 'IFCI',
 'ITC',
 'JSL',
 'NAT ALUM',
 'NBCC',
 'NHPC',
 'PFC',
 'POLYPLEX',
 'POWERGRID',
 'SBI',
 'TATA STEEL']

In [10]:
# set extension
ext = '.png'
results = []

for share in my_shares:
    
    share_ext = share + ext
    
    driver.get("https://www.google.com/search?q=" + share + ' share price')    
    print(share)
    
    path = actual_directory + '\\' + share_ext
    #print(path)
    
    # select results pane -> div with id 'search'
    element = driver.find_element_by_id("search")

    location = element.location
    size = element.size

    driver.save_screenshot(path)

    x = location['x']
    y = location['y']
    w = size['width']
    h = size['height']
    
    width = x + w
    height = y + h

    im = Image.open(path)
    im = im.crop((int(x), int(y), int(width), int(height)))
    mini_path = mini_directory + '\\' + share_ext
    im.save(mini_path)

    #print image_to_string(Image.open(mini_path))
    share_details = ocr_core(mini_path)
    share_name_extracted = share_details.split('\n')[0]
    share_price_ext = share_details.split('\n')[3]

    # 99.20 ine +0.35 (0.35%) +
    share_price = share_price_ext.split(' ')[0]
    try:
        share_price_change = share_price_ext.split(' ')[-2]
    except:
        share_price_change = 'Undefined'
        
    share_price_change = share_price_change.replace('(', '').replace(')', '')
    
    results.append([share, share_name_extracted, share_price_ext, share_price, share_price_change]) 

AB CAPITAL
ENGINEERS INDIA
IFCI
ITC
JINDAL STAIN
NAT ALUM
NBCC
NHPC
PFC
POLYPLEX
POWERGRID
SBI
TATA STEEL


In [25]:
performance = pd.DataFrame(results)
performance.columns = ['Share', 'Search', 'Extraction', 'Share Price', 'Change']
performance

,Share,Search,Extraction,Share Price,Change
0,AB CAPITAL,Aditya Birla Capital Ltd,95.25 INR -0.90 (0.94%) +,95.25,0.94%
1,ENGINEERS INDIA,Engineers India Limited,100.95 nr -0.10 (0.099%) +,100.95,0.099%
2,IFCI,IFCI Limited,6.70 INR +0.40 (6.35%) +,6.70,6.35%
3,ITC,ITC Ltd,238.65 ine -0.25 (0.10%) +,238.65,0.10%
4,JINDAL STAIN,Jindal Stainless Ltd,37.50 Inr -0.95 (2.47%) +,37.50,2.47%
5,NAT ALUM,National Aluminium Company Limited,42.85 inp +1.70 (4.13%) *,42.85,4.13%
6,NBCC,NBCC (India) Ltd,34.55 Inr -0.20 (0.58%) +,34.55,0.58%
7,NHPC,NHPC Limited,24.00 ine +0.20 (0.84%) +,24.00,0.84%
8,PFC,Power Finance Corporation Limited,111.95 nr +0.050 (0.045%) +,111.95,0.045%
9,POLYPLEX,Polyplex Corporation Limited,478.15 inr-0.45 (0.094%) +,478.15,0.094%


In [26]:
lock_stock[date_] = performance['Share Price']

In [28]:
writer = pd.ExcelWriter(file, engine='xlsxwriter')
lock_stock.to_excel(writer, sheet_name='LockStockBarrel')
performance.to_excel(writer, sheet_name=date_)
lock_inactive.to_excel(writer, sheet_name='Inactive')

writer.save()
writer.close()